In [1]:
from sage.algebras.flag_algebras import *

# This is some hack to create the theory for 3-graphs without C5- and K4-
# up to size 7. It is easier to make them as extensions of 6 sized structures
# so this code does that.

def check_containment(smalls, larges):
    """
    Helper function to check is any of the smalls appears in each of the larges.

    INPUT:
    smalls - list of flags, must be from a theory with edges relation
    larges - list of flags, also must be from a theory with edges relation

    OUTPUT:
    list of booleans, i-th element represents if i-th large flag is free from all smalls
    """
    sis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in smalls]
    lis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in larges]
    res = []
    for ll in lis:
        good = True
        for ss in sis:
            for _ in ll.isomorphic_substructures_iterator(ss):
                good = False
                break
            if not good:
                break
        res.append(good)
    return res

# Reset three graphs, so nothing is excluded
TG = ThreeGraphTheory
# k4 and k4m (the two induced structures with size 4 excluded)
k4 = TG(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3]])
k4m = TG(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3]])
excls = [k4,k4m]
TG.exclude(excls)

# check the list of flags with size 5 and 6
fl5 = TG.generate_flags(5)
fl6 = TG.generate_flags(6)

# quick 3-graph identifier code. This will be the identifier for
# the theory of C5- free 3-graphs (any identifier working for 3-graphs can work here)
def _identify_hypergraph(n, ftype_points, edges):
    g = Graph([list(range(n+len(edges))), [(i+n,x) for i,b in enumerate(edges) for x in b]], 
              format='vertices_and_edges')
    ftype_union = [jj for ff in ftype_points for jj in ff]
    partt = list(ftype_points) + \
            [[ii for ii in range(n) if ii not in ftype_union]] + \
            [list(range(n,n+len(edges)))]
    blocks = tuple(g.canonical_label(partition=partt).edges(labels=None, sort=True))
    return (n, tuple([len(xx) for xx in ftype_points]), blocks)

# generator code. It should really just return TG, but for size 7 that takes too long
# so this hack just returns TG for size up to 6, and for 7 it generates all flags
# with this extension technique
def _gen(n):
    if n<=4:
        for xx in TG.generate_flags(n):
            yield xx.blocks()
    elif n==5:
        for xx in fl5:
            yield xx.blocks()
    elif n==6:
        for xx in fl6:
            yield xx.blocks()
    elif n==7:
        import itertools
        from tqdm import tqdm
        fl7_m = [[] for ii in range(35+1)]
        subs = list(itertools.combinations(range(6), int(2)))
        for xx in tqdm(fl6):
            xb = xx.blocks()['edges']
            for ii in range(15+1):
                for pps in itertools.combinations(subs, int(ii)):
                    xbp = [[pp[0], pp[1], 6] for pp in pps] + xb
                    flxp = TG(7, edges=xbp)
                    en = len(xbp)
                    if flxp not in fl7_m[en]:
                        if check_containment(excls, [flxp])[0]:
                            fl7_m[en].append(flxp)
        fl7 = [yy for xx in fl7_m for yy in xx]
        for xx in fl7:
            yield xx.blocks()
    else:
        #for n>=8 just return an empty list, this will not be called so doesn't 
        #really matter
        return []

# Create the theory based on this generator and identifier
TGp = CombinatorialTheory("NoK4m", _gen, _identify_hypergraph, edges=3)

# for sanity check, print the number of structures with size 5, 6, 7
# should be 11 106 8157
# print(len(TGp.generate_flags(5)), len(TGp.generate_flags(6)), len(TGp.generate_flags(7)))

In [2]:
tp = TGp(3, ftype=[0, 1, 2], edges=[[0, 1, 2]])
all_6 = TGp.generate_flags(6, tp)
all_6_flags = []

for f in all_6:
    # Re-label vertices to match type [0, 1, 2]
    roots = f.ftype_points()
    edges = f.blocks()['edges'].copy()
    isom = [-1 for i in range(6)]
    for i in range(3):
        isom[ roots[i] ] = i
    idx = 3
    for i in range(6):
        if isom[i] == -1:
            isom[i] = idx
            idx += 1
    for i in range(len(edges)):
        for j in range(len(edges[i])):
            edges[i][j] = isom[ edges[i][j] ]
        edges[i] = sorted(edges[i])
        
    # Re-labeled flag
    flag = TGp(6, ftype=[0, 1, 2], edges=edges)
    
    # Only preserve flags that contain edge [3, 4, 5]
    if [3, 4, 5] not in flag.blocks()['edges']:
        continue

    # Get pattern of the top vertices
    states = [[], [], []]
    for v in [3, 4, 5]:
        for e in edges:
            if v in e:
                t = [x for x in e if x <= 2]
                if len(t) > 1:
                    states[v - 3] += t
    for i in range(3):
        states[i] = list({0, 1, 2}.difference(set(states[i])))
        if len(states[i]) == 1:
            states[i] = states[i][0]
        elif len(states[i]) == 2:
            print("FATAL ERROR")
        else:
            states[i] = -1

    all_6_flags.append([flag, states])

In [3]:
# Optimisation conditions
degree = TGp(3, edges=[[0,1,2]], ftype=[0])
p2f4 = TGp.generate_flags(4, TGp(2, ftype=[0, 1]))
degree_difference = p2f4[2]-p2f4[3]+p2f4[5]-p2f4[6]
positives = [degree-2/7, degree_difference, -degree_difference]

In [5]:
T_3 = 1
T_21 = 1
T_111 = 1
for flag, states in all_6_flags:
    if states.count(-1) == 0:
        if len(set(states)) == 2:
            T_21 += flag
        elif len(set(states)) == 3:
            T_111 += flag
        elif len(set(states)) == 1:
            T_3 += flag
    elif states.count(-1) == 1:
        if len(set(states)) == 2:
            T_3 += flag / 3
            T_21 += flag * 2 / 3
        elif len(set(states)) == 3:
            T_21 += flag * 2 / 3
            T_111 += flag / 3
    elif states.count(-1) in [2, 3]:
        T_3 += flag / 9
        T_21 += flag * 2 / 3
        T_111 += flag * 2 / 9
T_3 -= 1
T_21 -= 1
T_111 -= 1

In [6]:
# a_3 <= 0.02148518586113618
ans = TGp.optimize_problem(T_3, 6, maximize=True, positives=positives)
print(ans)

Base flags generated, their number is 106
The relevant ftypes are constructed, their number is 4
Block sizes before symmetric/asymmetric change is applied: [8, 41, 26, 18]


Done with mult table for Ftype on 4 points with edges=[[0, 1, 2], [0, 1, 3]]: : 4it [00:00, 3910.77it/s]


Tables finished


Done with positivity constraint 2: 100%|██████████████████████| 3/3 [00:00<00:00, 155.13it/s]

Constraints finished
Running sdp without construction. Used block sizes are [6, 2, 7, 34, 8, 18, 9, 9, -106, -23]


CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 8.27e-01 Pobj: -6.1221373e+01 Ad: 3.11e-01 Dobj: -4.4003798e-02 
Iter:  2 Ap: 1.00e+00 Pobj: -9.5075749e+01 Ad: 7.03e-01 Dobj:  2.8520277e-02 
Iter:  3 Ap: 1.00e+00 Pobj: -9.8754754e+01 Ad: 9.08e-01 Dobj: -1.8212967e-02 
Iter:  4 Ap: 1.00e+00 Pobj: -9.7520022e+01 Ad: 7.12e-01 Dobj: -2.0977447e-02 
Iter:  5 Ap: 5.20e-01 Pobj: -6.7857962e+01 Ad: 4.32e-01 Dobj: -2.5354313e-02 
Iter:  6 Ap: 5.88e-01 Pobj: -5.7967640e+01 Ad: 7.04e-01 Dobj: -2.7208688e-02 
Iter:  7 Ap: 1.00e+00 Pobj: -3.6259070e+01 Ad: 5.77e-01 Dobj: -2.2384378e-02 
Iter:  8 Ap: 1.00e+00 Pobj: -2.2407585e+01 Ad: 6.20e-01 Dobj: -1.9497151e-02 
Iter:  9 Ap: 8.83e-01 Pobj: -1.9446224e+01 Ad: 5.92e-01 Dobj: -1.5009041e-02 
Iter: 10 Ap: 4.59e-01 Pobj: -1.6926025e+01 Ad: 5.74e-01 Dobj: -1.6143363e-02 
Iter: 11 Ap: 5.57e-01 Pobj: -1.3058661e+01 Ad: 6.92e-01 Dobj: -1.5095554e-02 
Iter: 12 Ap: 8.63e-01 Pobj: -6.1976382e+00 Ad: 9.31e-

In [6]:
T_111 = 1
for flag, states in all_6_flags:
    if -1 not in states and len(set(states)) == 1:
        continue
    if states.count(-1) == 1 and len(set(states)) == 2:
        continue
    # Add flags with coefficients
    if states.count(-1) == 2 and len(set(states)) == 2:
        T_111 += flag * 2 / 9
    elif states.count(-1) == 3:
        T_111 += flag * 2 / 9
    elif states.count(-1) == 1 and len(set(states)) == 3:
        T_111 += flag / 3
    else:
        T_111 += flag
    # print(flag)
    # print(states)
T_111 -= 1

# a_111 >= 0.20655245288809151
ans = TGp.optimize_problem(T_111, 6, maximize=False, positives=positives)
print(ans)

Base flags generated, their number is 106
The relevant ftypes are constructed, their number is 4
Block sizes before symmetric/asymmetric change is applied: [8, 41, 26, 18]


Done with mult table for Ftype on 4 points with edges=[[0, 1, 2], [0, 1, 3]]: : 4it [00:00, 2747.21it/s]


Tables finished


Done with positivity constraint 2: 100%|█████████| 3/3 [00:00<00:00, 124.67it/s]

Constraints finished
Running sdp without construction. Used block sizes are [6, 2, 7, 34, 8, 18, 9, 9, -106, -23]


CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 8.12e-01 Pobj: -6.2696503e+01 Ad: 2.96e-01 Dobj:  5.1546306e-01 
Iter:  2 Ap: 1.00e+00 Pobj: -1.0165115e+02 Ad: 6.80e-01 Dobj: -2.9137775e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -1.0617927e+02 Ad: 9.07e-01 Dobj:  8.0050121e-02 
Iter:  4 Ap: 1.00e+00 Pobj: -1.0582061e+02 Ad: 7.32e-01 Dobj:  1.2110777e-01 
Iter:  5 Ap: 6.45e-01 Pobj: -7.2783706e+01 Ad: 4.77e-01 Dobj:  1.3952678e-01 
Iter:  6 Ap: 5.96e-01 Pobj: -6.2891914e+01 Ad: 7.08e-01 Dobj:  1.8759147e-01 
Iter:  7 Ap: 1.00e+00 Pobj: -3.9396699e+01 Ad: 5.79e-01 Dobj:  2.0380229e-01 
Iter:  8 Ap: 1.00e+00 Pobj: -2.4733827e+01 Ad: 5.94e-01 Dobj:  2.1442186e-01 
Iter:  9 Ap: 1.00e+00 Pobj: -2.0894941e+01 Ad: 6.34e-01 Dobj:  2.2483981e-01 
Iter: 10 Ap: 6.28e-01 Pobj: -1.5866348e+01 Ad: 6.03e-01 Dobj:  2.2464449e-01 
Iter: 11 Ap: 7.54e-01 Pobj: -1.1130172e+01 Ad: 7.58e-01 Dobj:  2.2900507e-01 
Iter: 12 Ap: 9.70e-01 Pobj: -4.1390824e+00 Ad: 8.82e-

In [7]:
T_21 = 1
for flag, states in all_6_flags:
    if -1 not in states and len(set(states)) == 1:
        continue
    if -1 not in states and len(set(states)) == 3:
        continue
    # Add flags with coefficients
    if -1 not in states and len(set(states)) == 2:
        T_21 += flag
    else:
        T_21 += flag * 2 / 3
    # print(flag)
    # print(states)
T_21 -= 1

# a_21 >= 0.15849272791779434
ans = TGp.optimize_problem(T_21, 6, maximize=False, positives=positives)
print(ans)

Base flags generated, their number is 106
The relevant ftypes are constructed, their number is 4
Block sizes before symmetric/asymmetric change is applied: [8, 41, 26, 18]


Done with mult table for Ftype on 4 points with edges=[[0, 1, 2], [0, 1, 3]]: : 4it [00:00, 2789.69it/s]


Tables finished


Done with positivity constraint 2: 100%|████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 130.29it/s]

Constraints finished
Running sdp without construction. Used block sizes are [6, 2, 7, 34, 8, 18, 9, 9, -106, -23]


CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 8.16e-01 Pobj: -6.2177999e+01 Ad: 3.00e-01 Dobj:  2.0983211e-01 
Iter:  2 Ap: 1.00e+00 Pobj: -1.0011557e+02 Ad: 6.84e-01 Dobj: -2.3756958e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -1.0439123e+02 Ad: 9.07e-01 Dobj:  1.0224201e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -1.0369049e+02 Ad: 7.30e-01 Dobj:  1.3193237e-01 
Iter:  5 Ap: 6.25e-01 Pobj: -7.0011362e+01 Ad: 4.58e-01 Dobj:  1.2215432e-01 
Iter:  6 Ap: 6.44e-01 Pobj: -6.1231889e+01 Ad: 7.17e-01 Dobj:  1.8248204e-01 
Iter:  7 Ap: 1.00e+00 Pobj: -3.8777332e+01 Ad: 6.06e-01 Dobj:  1.9960575e-01 
Iter:  8 Ap: 9.78e-01 Pobj: -2.4482788e+01 Ad: 5.58e-01 Dobj:  2.0023251e-01 
Iter:  9 Ap: 1.00e+00 Pobj: -2.1980691e+01 Ad: 6.44e-01 Dobj:  2.0242710e-01 
Iter: 10 Ap: 6.19e-01 Pobj: -1.5986677e+01 Ad: 5.85e-01 Dobj:  2.0253008e-01 
Iter: 11 Ap: 7.16e-01 Pobj: -1.1773399e+01 Ad: 7.47e-01 Dobj:  2.0289161e-01 
Iter: 12 Ap: 9.38e-01 Pobj: -4.5830784e+00 Ad: 8.77e-

In [8]:
# (a_111 + a_21) / 2 >= 0.1995627514370104
ans = TGp.optimize_problem((T_111 + T_21) / 2, 6, maximize=False, positives=positives)
print(ans)

Base flags generated, their number is 106
The relevant ftypes are constructed, their number is 4
Block sizes before symmetric/asymmetric change is applied: [8, 41, 26, 18]


Done with mult table for Ftype on 4 points with edges=[[0, 1, 2], [0, 1, 3]]: : 4it [00:00, 2932.57it/s]


Tables finished


Done with positivity constraint 2: 100%|█████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 99.40it/s]

Constraints finished
Running sdp without construction. Used block sizes are [6, 2, 7, 34, 8, 18, 9, 9, -106, -23]


CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 8.15e-01 Pobj: -6.2149239e+01 Ad: 3.00e-01 Dobj:  3.6711944e-01 
Iter:  2 Ap: 1.00e+00 Pobj: -1.0013010e+02 Ad: 6.84e-01 Dobj: -2.6165721e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -1.0440563e+02 Ad: 9.07e-01 Dobj:  9.1611093e-02 
Iter:  4 Ap: 1.00e+00 Pobj: -1.0371217e+02 Ad: 7.30e-01 Dobj:  1.2670513e-01 
Iter:  5 Ap: 6.24e-01 Pobj: -7.0013279e+01 Ad: 4.58e-01 Dobj:  1.3085623e-01 
Iter:  6 Ap: 6.47e-01 Pobj: -6.1210816e+01 Ad: 7.17e-01 Dobj:  1.8576552e-01 
Iter:  7 Ap: 1.00e+00 Pobj: -3.8811474e+01 Ad: 6.09e-01 Dobj:  2.0231242e-01 
Iter:  8 Ap: 9.72e-01 Pobj: -2.4564294e+01 Ad: 5.53e-01 Dobj:  2.0752101e-01 
Iter:  9 Ap: 1.00e+00 Pobj: -2.2111126e+01 Ad: 6.44e-01 Dobj:  2.1413716e-01 
Iter: 10 Ap: 6.14e-01 Pobj: -1.6113163e+01 Ad: 5.86e-01 Dobj:  2.1359956e-01 
Iter: 11 Ap: 7.01e-01 Pobj: -1.2006106e+01 Ad: 7.42e-01 Dobj:  2.1582460e-01 
Iter: 12 Ap: 9.31e-01 Pobj: -4.7127848e+00 Ad: 8.78e-